# Yolo-v2 DW2TF validation

In [1]:
import numpy as np
import tensorflow as tf

# Random input data to test 
in_data = np.random.randn(64, 608, 608, 3) * 120.56

### Output from Darkflow converted Yolo-v2

In [2]:
tf.reset_default_graph()

saver = tf.train.import_meta_graph('/data/darkflow/yolov2.ckpt.meta')
ckpt_path = '/data/darkflow/yolov2.ckpt'

g = tf.get_default_graph()
in_t = g.get_tensor_by_name('input:0')
out_t = g.get_tensor_by_name('output:0')

with tf.Session() as sess:
    saver.restore(sess, ckpt_path)
    
    out_data_darkflow = sess.run(out_t, feed_dict={in_t: in_data})

W0730 11:42:36.966557 140635950143232 deprecation.py:323] From /scratch/anaconda3/envs/tf1.14/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


### Output from DW2TF converted Yolo-v2

In [3]:
tf.reset_default_graph()

saver = tf.train.import_meta_graph('/data/DW2TF/data/yolov2.ckpt.meta')
ckpt_path = '/data/DW2TF/data/yolov2.ckpt'

g = tf.get_default_graph()
in_t = g.get_tensor_by_name('yolov2/net1:0')
out_t = g.get_tensor_by_name('yolov2/convolutional23/BiasAdd:0')

with tf.Session() as sess:
    saver.restore(sess, ckpt_path)
    
    out_data_dw2tf = sess.run(out_t, feed_dict={in_t: in_data})

### RMS Error

In [4]:
print(out_data_darkflow.shape)
print(out_data_dw2tf.shape)
print(np.mean(out_data_darkflow - out_data_dw2tf)**2)

(64, 19, 19, 425)
(64, 19, 19, 425)
5.125400967058772e-15


In [5]:
print(out_data_darkflow[0,0,0,:10])

[ -4.4880652   5.9635143  -4.9752836  -2.7621126 -20.442324   14.926774
  18.849686   13.160991    3.2470438 -14.896617 ]


In [6]:
print(out_data_dw2tf[0,0,0,:10])

[ -4.4880676   5.963505   -4.9753103  -2.7621546 -20.442469   14.926693
  18.849705   13.1609535   3.2470534 -14.8967   ]
